In [0]:
#Uncomment line below and run if you need to clear all your resources
#!kill -9 -1

In [0]:
#Installing necessary libraries, will need to do everytime you start the vm

In [0]:
#Upgrade to the latest scipy to get rid of a warning message in later installs
!pip install --upgrade scipy --quiet

In [0]:
#Install the version of pytorch required for fastai (0.4.0 also works but you get warning messages)
!pip install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl --quiet && pip install torchvision --quiet

In [0]:
#Instal fastai
!pip install fastai --quiet

In [0]:
#RESTART RUNTIME (Runtime->Restart runtime) AFTER COMPLETING THE ABOVE INSTALLS

In [0]:
#Now we need to download the data

In [0]:
#Start by installing the kaggle command line interface

In [0]:
!pip install kaggle-cli --quiet

In [0]:
#Configure the kaggle command line inteface to with your username, password and competition

In [0]:
!kg config -g -u esteban.j.guillen@gmail.com -p wildcats -c planet-understanding-the-amazon-from-space

In [0]:
#Create a directory to store our data

In [0]:
!mkdir planet-data

In [0]:
#Download the training data

In [7]:
!kg download -f train-jpg.tar.7z

/usr/local/lib/python3.6/dist-packages/mechanicalsoup/browser.py:37: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 37 of the file /usr/local/lib/python3.6/dist-packages/mechanicalsoup/browser.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  response.content, **soup_config)
downloading https://www.kaggle.com/c/planet-understanding-the-amazon-from-space/download/train-jpg.tar.7z

train-jpg.tar.7z 100% |#############################| Time: 0:00:21  28.4 MiB/s



In [0]:
#Uncompress the data

In [9]:
!7za x train-jpg.tar.7z


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 629296207 bytes (601 MiB)

Extracting archive: train-jpg.tar.7z
--
Path = train-jpg.tar.7z
Type = 7z
Physical Size = 629296207
Headers Size = 98
Method = LZMA2:24
Solid = -
Blocks = 1

  0%      0% - train-jpg.tar                      1% - train-jpg.tar                      2% - train-jpg.tar                      3% - train-jpg.tar                      4% - train-jpg.tar                      5% - train-jpg.tar                      6% - train-jpg.tar

In [0]:
#Unpack the data to our data directory

In [0]:
!tar xf train-jpg.tar --directory planet-data/

In [0]:
#Download the training labels

In [13]:
!kg download -f train_v2.csv.zip

/usr/local/lib/python3.6/dist-packages/mechanicalsoup/browser.py:37: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 37 of the file /usr/local/lib/python3.6/dist-packages/mechanicalsoup/browser.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  response.content, **soup_config)
downloading https://www.kaggle.com/c/planet-understanding-the-amazon-from-space/download/train_v2.csv.zip

train_v2.csv.zip 100% |#############################| Time: 0:00:00 813.7 KiB/s



In [0]:
#Unpack to the data directory

In [15]:
!unzip train_v2.csv.zip -d planet-data

Archive:  train_v2.csv.zip
  inflating: planet-data/train_v2.csv  
   creating: planet-data/__MACOSX/
  inflating: planet-data/__MACOSX/._train_v2.csv  


In [0]:
#Allows our plots to show up in the notebook

In [0]:
%matplotlib inline

In [0]:
#Import the necessary fastai libraries
from fastai.conv_learner import *
from fastai.plots import *

In [0]:
#Points to our data directory where the training data and labels are located
PATH = 'planet-data/'

In [0]:
from sklearn.metrics import *

def f2(preds, targs, start=0.17, end=0.24, step=0.01):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        return max([fbeta_score(targs, (preds>th), 2, average='samples')
                    for th in np.arange(start,end,step)])


In [21]:
list_paths = [f"{PATH}train-jpg/train_0.jpg", f"{PATH}train-jpg/train_1.jpg",f"{PATH}train-jpg/train_12.jpg"]
titles=["haze primary", "agriculture clear primary water", "cloudy"]
plots_from_files(list_paths, titles=titles, maintitle="Multi-label classification")

In [0]:
#Our pretrained neural net architecture
f_model = resnet34

#The performance measure we will use to evaluate our model
metrics=[f2]

In [0]:
#Point to the file that contains our labels (images -> [list of labels])
label_csv = f'{PATH}train_v2.csv'

#Determine how many images are in the training set (first row is column header)
n = len(list(open(label_csv)))-1

#Identify which images will be part of the validation set (80% train, 20% val)
val_idxs = get_cv_idxs(n)

In [0]:
#Helper method to 
def get_data(sz):
    tfms = tfms_from_model(f_model, sz, aug_tfms=transforms_top_down, max_zoom=1.05)
    return ImageClassifierData.from_csv(PATH, 'train-jpg', label_csv, tfms=tfms,
                    suffix='.jpg', val_idxs=val_idxs)

In [0]:
#This will control how images are scaled before the are passed into the network
sz=64

In [0]:
#Load our data
data = get_data(sz)

In [27]:
#Create a pretrained model from the architecture we specified
learn = ConvLearner.pretrained(f_model, data, metrics=metrics, ps=0.5, precompute=True)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /content/.torch/models/resnet34-333f7ec4.pth
100%|██████████| 87306240/87306240 [00:03<00:00, 24604611.78it/s]


100%|██████████| 127/127 [00:20<00:00,  6.10it/s]


In [28]:
learn.summary

<bound method ConvLearner.summary of Sequential(
  (0): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True)
  (1): Dropout(p=0.5)
  (2): Linear(in_features=1024, out_features=512)
  (3): ReLU()
  (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True)
  (5): Dropout(p=0.5)
  (6): Linear(in_features=512, out_features=17)
  (7): Sigmoid()
)>

In [0]:
#Define the learning rate
lr = 0.2

In [32]:
#Train for 5 epochs
learn.fit(lr, 5, cycle_len=1)

epoch      trn_loss   val_loss   f2         
    0      0.150705   0.135808   0.880209  
    1      0.140971   0.130171   0.886658  
    2      0.138751   0.127257   0.890053  
    3      0.133554   0.125851   0.891411  
    4      0.136688   0.12482    0.892372  



[array([0.12482]), 0.8923723870673956]

In [0]:
#Define learning rated for different parts of our model
lrs = np.array([lr/9,lr/3,lr])

In [34]:
#Important to turn precompute off so we can now take advantage of data augmentation
learn.precompute=False

#Allow the pretrained network to start learning (weights get updated)
learn.unfreeze()

#Train for 3 epochs (first cycle is of lenght 1 epoch, second cycle is of length 2 epochs)
learn.fit(lrs, 2, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss   f2         
    0      0.1204     0.106277   0.911677  
    1      0.109369   0.101384   0.916049  
    2      0.106284   0.098223   0.918056  



[array([0.09822]), 0.918055755670647]

In [0]:
#Test using the validation set and used Test Time Augmentation
multi_preds, y = learn.TTA()
preds = np.mean(multi_preds, 0)

In [0]:
#Make final prediction
f2(preds,y)

0.9179947311508401